# Book Recommender System 

## 狄豪飛 111550196  ## 邱倫恩 111550195 


In [4]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Load the datasets
books_path = 'Books.csv'
users_path = 'Users.csv'
ratings_path = 'Ratings.csv'

# Database connection parameters
db_endpoint = "booksdatabase.cae1sdou4ahe.us-east-1.rds.amazonaws.com"
db_port = "5432"
db_name = "postgres"  # Replace with your actual database name
db_user = "postgres"
db_password = "12345678"

# Create a connection URL
conn_url = f"postgresql://{db_user}:{db_password}@{db_endpoint}:{db_port}/{db_name}"

# Create an engine to manage the connection
engine = create_engine(conn_url)

try:
    # Replace your pd.read_csv() calls with pd.read_sql() calls
    books_df = pd.read_sql("SELECT * FROM books", engine)
    users_df = pd.read_sql("SELECT * FROM users", engine)
    ratings_df = pd.read_sql("SELECT * FROM ratings", engine)

    # Display the first few rows of each dataset
    books_head = books_df.head()
    users_head = users_df.head()
    ratings_head = ratings_df.head()
except Exception as e:
    error_message = str(e)

books_head, users_head, ratings_head, error_message if 'error_message' in locals() else "No errors"

(         isbn                                 Book-Title  \
 0  0312874278                 Through Wolf's Eyes (Wolf)   
 1  084394899X                               Eagle Dancer   
 2  0505524481        Across a Moonswept Moor (Timeswept)   
 3  0449005887                               The Last Man   
 4  1879941066  Georgia Scenes (Southern Classics Series)   
 
                    Book-Author Year-Of-Publication  \
 0               Jane Lindskold                2001   
 1                Theresa Scott                2001   
 2                Julie Moffett                2001   
 3               Charles Kenney                2001   
 4  Augustus Baldwin Longstreet                1992   
 
                        publisher  \
 0                      Tor Books   
 1                  Leisure Books   
 2  Dorchester Publishing Company   
 3               Ballantine Books   
 4      J. S. Sanders and Company   
 
                                          Image-URL-S  \
 0  http://images.a

### Book dataset (Books.csv):

#### Columns: ISBN, Book-Title, Book-Author, Year-Of-Publication, Publisher, Image-URL-S, Image-URL-M, Image-URL-L.
First Lines:
ISBN: 0195153448, Book-Title: Classical Mythology, Author: Mark P. O. Morford, etc.
Note: Some columns, such as Year-Of-Publication, appear to have mixed data types.

### User data set (Users.csv):

#### Columns: User-ID, Location, Age.
First Rows:
User-ID: 1, Location: nyc, new york, usa, Age: NaN, etc.
Note: The Age column contains missing values (NaN).

### Ratings Dataset (Ratings.csv):

#### Columns: User-ID, ISBN, Book-Rating.
First Rows:
User-ID: 276725, ISBN: 034545104X, Book-Rating: 0, etc.

## Let's proceed with cleaning and preparing the data. This process involves handling missing values, outliers, and normalizing formats. I'll address each dataset separately:

### Books Dataset:

Handle mixed data types in Year-Of-Publication.
Verify and clean publisher and author fields.
Exclude books with insufficient information.
Users Dataset:

Handle missing values in Age.
Normalize the Location field, if necessary.
Ratings Dataset:

Validate the ISBN references to ensure they match with the books dataset.
Check for any anomalies in the Book-Rating field.
I'll start with the books dataset, focusing on the Year-Of-Publication field. Let's clean and transform this data first.


In [7]:
# Cleaning the Books Dataset

# Convert Year-Of-Publication to numeric, set errors to NaN
books_df['Year-Of-Publication'] = pd.to_numeric(books_df['Year-Of-Publication'], errors='coerce')

# Replace invalid years with NaN (e.g., years in the future or too far in the past)
current_year = pd.Timestamp.now().year
books_df.loc[(books_df['Year-Of-Publication'] > current_year) | (books_df['Year-Of-Publication'] < 1800), 'Year-Of-Publication'] = None

# Check for missing values in important columns
missing_values_books = books_df.isnull().sum()

# Drop rows where important information is missing
books_df_cleaned = books_df.dropna(subset=['Book-Title', 'Book-Author', 'Year-Of-Publication', 'publisher'])

missing_values_books, books_df_cleaned.head()

(isbn                      0
 Book-Title                0
 Book-Author               1
 Year-Of-Publication    4635
 publisher                 2
 Image-URL-S               0
 Image-URL-M               0
 Image-URL-L               3
 dtype: int64,
          isbn                                 Book-Title  \
 0  0312874278                 Through Wolf's Eyes (Wolf)   
 1  084394899X                               Eagle Dancer   
 2  0505524481        Across a Moonswept Moor (Timeswept)   
 3  0449005887                               The Last Man   
 4  1879941066  Georgia Scenes (Southern Classics Series)   
 
                    Book-Author  Year-Of-Publication  \
 0               Jane Lindskold               2001.0   
 1                Theresa Scott               2001.0   
 2                Julie Moffett               2001.0   
 3               Charles Kenney               2001.0   
 4  Augustus Baldwin Longstreet               1992.0   
 
                        publisher  \
 0        

In [9]:
# Cleaning the Users Dataset

# Handling missing values in 'Age'
# Replace invalid ages (e.g., less than 5 and greater than 100) with NaN
users_df['age'] = users_df['age'].apply(lambda x: x if 5 <= x <= 100 else None)

# Check for missing values
missing_values_users = users_df.isnull().sum()

# For this example, we'll leave the NaN values as is, as imputation might not be accurate
# and removing all rows with missing ages could significantly reduce the dataset size

# Normalizing the 'Location' field is optional and can be complex. 
# It might involve parsing the location string and standardizing it.
# For simplicity, we'll leave it as is for now.

missing_values_users, users_df.head()

(User-ID          0
 location         0
 age         112010
 dtype: int64,
    User-ID                            location   age
 0        1                  nyc, new york, usa   NaN
 1        2           stockton, california, usa  18.0
 2        3     moscow, yukon territory, russia   NaN
 3        4           porto, v.n.gaia, portugal  17.0
 4        5  farnborough, hants, united kingdom   NaN)

In [11]:
# Cleaning the Ratings Dataset

# Validate the ISBN references
# Keep only those ratings where the ISBN exists in the books dataset
ratings_df_cleaned = ratings_df[ratings_df['isbn'].isin(books_df_cleaned['isbn'])]

# Check for any anomalies in the 'Book-Rating' field
# Assuming ratings should be within a specific range (e.g., 0-10)
ratings_df_cleaned = ratings_df_cleaned[(ratings_df_cleaned['Book-Rating'] >= 0) & (ratings_df_cleaned['Book-Rating'] <= 10)]

# Check for missing values and the size of the cleaned dataset
missing_values_ratings = ratings_df_cleaned.isnull().sum()
cleaned_ratings_size = ratings_df_cleaned.shape

missing_values_ratings, cleaned_ratings_size

(User-ID        0
 isbn           0
 Book-Rating    0
 dtype: int64,
 (1017065, 3))

### The cleaning of the Books dataset has been completed with the following steps:

- Converted Year-Of-Publication to numeric, setting non-numeric values to NaN.
- Replaced invalid years (those in the future or before 1800) with NaN.
- Checked for missing values in key columns (Book-Author, Year-Of-Publication, Publisher). We found:
- Book-Author: 1 missing value
- Year-Of-Publication: 4,635 missing values
- Publisher: 2 missing values
- Dropped rows with missing values in these critical columns.

### The cleaning of the Users dataset has been done with these steps:

- Invalid ages (less than 5 or greater than 100) have been replaced with NaN.
- There are 112,010 missing values in the Age field.
- We chose not to impute ages due to potential inaccuracies, and because removing all rows with missing ages could significantly reduce the dataset.
- The Location field has been left as is for now, as normalization can be complex and may not be crucial for the recommendation system.

Then, we clean the Ratings dataset. This involves validating ISBN references against the books dataset and checking for anomalies in the Book-Rating field. Let's proceed with that.

### The cleaning of the Ratings dataset has been completed with the following steps:

- Validated ISBN references, ensuring all ratings correspond to books present in the cleaned Books dataset.
- Checked for anomalies in Book-Rating, confirming all ratings are within the expected range (0-10).
- No missing values were found in the cleaned Ratings dataset.
- The size of the cleaned Ratings dataset is 1,017,065 rows.

# Developing the Recommendation Model.

In our book recommendation system, we're employing a method known as Collaborative Filtering (CF). This technique makes automatic predictions (filtering) about the interests of a user by collecting preferences from many users (collaborating). The underlying assumption of CF is that if a person A has the same opinion as a person B on an issue, A is more likely to have B's opinion on a different issue than that of a random person.

##### Collaborative Filtering Overview:

- #### User-Item Interactions: 
Our model is based on user-item interactions, which in our case are the book ratings given by users. We don't need any information about the books or the users themselves (like book genres or user demographics) — we only need to know who rated what and how much they liked it.

- #### Similarity of Preferences:
The core idea is to find users who have similar rating patterns. If user A rates books X and Y highly, and user B does the same, then we can infer they have similar tastes. If user A has rated book Z highly, but user B hasn't rated it yet, we might predict that user B will also like book Z.

- #### Matrix of Ratings: 
We arrange our data into a matrix where each row represents a user and each column represents a book. The entries of this matrix are the ratings that users have given to books.

##### Here’s how our model works:

- #### Data Matrix: 
We start with a matrix where each row corresponds to a user, and each column corresponds to an item (in our case, books). The matrix entries are the ratings users have given to the books.

- #### Similarity Assessment: 
The model looks for similarities between users based on their ratings. If two users have rated a set of books similarly, the model infers that they have similar tastes.

- #### Predicting Ratings:
For a given user, the model predicts how they would rate books they haven't yet interacted with by aggregating the ratings of similar users. These predictions are based on the weighted average of the ratings from similar users, adjusted for the active user's overall rating behavior.

##### When we say 'Matrix Factorization', we're referring to:

- #### Matrix Decomposition:
This is a process where the original large matrix of user-item interactions is decomposed into two or more smaller matrices. The goal is to capture the underlying structure in the user-item matrix.

- #### Latent Factors:
These smaller matrices represent latent factors - hidden characteristics that might determine how a user rates an item. For books, latent factors could include genres or themes that are not explicitly labeled but can be inferred from user ratings.

- #### Learning and Optimization:
The model learns these latent factors by minimizing the difference between the observed ratings in the user-item matrix and the product of the latent factor matrices. Techniques like Stochastic Gradient Descent (SGD) or Alternating Least Squares (ALS) can be used for this optimization task.

- #### Prediction:
Once the model has learned these latent factors, it can predict a rating for a user-item pair where the rating was not previously known. It does this by taking the dot product of the latent factors for the user and the item.

- #### Recommendations:
Finally, the model can recommend items to a user by selecting items with the highest predicted ratings that the user has not yet rated.



### Step 1: Filter the Data

#### First, we need to filter the data according to the criteria:

* Include only users who have rated at least 50 books.
* Include only books that have been rated by at least 50 users.

In [13]:
# Reattempting Step 1: Filter the Data

# Filtering users who have rated at least 50 books
users_with_50_ratings = ratings_df_cleaned['User-ID'].value_counts()
users_with_50_ratings = users_with_50_ratings[users_with_50_ratings >= 50].index.tolist()

filtered_ratings = ratings_df_cleaned[ratings_df_cleaned['User-ID'].isin(users_with_50_ratings)]

# Filtering books that have been rated by at least 50 users
books_with_50_ratings = filtered_ratings['isbn'].value_counts()
books_with_50_ratings = books_with_50_ratings[books_with_50_ratings >= 50].index.tolist()

filtered_ratings = filtered_ratings[filtered_ratings['isbn'].isin(books_with_50_ratings)]

# Checking the shape of the filtered ratings dataset
filtered_ratings.shape

(98178, 3)

### Step 2: Build the Item Similarity Matrix

We will calculate the similarity between items (books) using the user ratings. Commonly used methods for similarity include cosine similarity, Pearson correlation, or Jaccard similarity. We'll select one based on the nature of our dataset.

The Item Similarity Matrix has been successfully built using cosine similarity. This matrix represents the similarity between each pair of books based on user ratings. In the matrix, each row and column correspond to a book's ISBN, and the values represent the similarity score between books.

The portion displayed shows the similarity scores for the first five books. A score of 1 indicates identical ratings patterns, while a score of 0 indicates no similarity.

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Creating a pivot table for the similarity matrix
pivot_table = filtered_ratings.pivot_table(index='isbn', columns='User-ID', values='Book-Rating').fillna(0)

# Calculating cosine similarity between items (books)
cosine_sim = cosine_similarity(pivot_table)

# Creating a DataFrame for the similarity matrix for better readability
similarity_matrix = pd.DataFrame(cosine_sim, index=pivot_table.index, columns=pivot_table.index)

# Displaying a portion of the similarity matrix
similarity_matrix.iloc[:5, :5]

isbn,002026478X,002542730X,0060008032,0060085444,0060096195
isbn,,,,,
002026478X,1.000000,0.000000,0.000000,0.000000,0.099748
002542730X,0.000000,1.000000,0.028303,0.000000,0.000000
0060008032,0.000000,0.028303,1.000000,0.054638,0.039401
0060085444,0.000000,0.000000,0.054638,1.000000,0.044737
0060096195,0.099748,0.000000,0.039401,0.044737,1.000000


### Step 3: Generating Book Recommendations
##### In this step, we'll use the Item Similarity Matrix to generate book recommendations. 

#### The process involves:

* #### Selecting a book as input (either randomly or based on a specific ISBN).
* #### Finding similar books based on the similarity matrix.
* #### Ranking these similar books to recommend the most similar ones.


In [15]:
def get_book_title(isbn, books_df):
    """ Get the book title given an ISBN. """
    title = books_df.loc[books_df['isbn'] == isbn, 'Book-Title'].iloc[0]
    return title

def recommend_books(isbn, similarity_matrix, books_df, num_recommendations=5):
    """
    Recommend books based on a given ISBN using the item similarity matrix.
    """
    # Retrieve similar books
    similar_books = similarity_matrix[isbn].sort_values(ascending=False)

    # Remove the book itself from the recommendation
    similar_books = similar_books[similar_books.index != isbn]

    # Get the top N similar books
    top_similar_books = similar_books.head(num_recommendations).index.tolist()

    # Get book titles
    recommended_books = [get_book_title(book_isbn, books_df) for book_isbn in top_similar_books]

    return recommended_books

### I'll demonstrate this by choosing a book from our dataset and then generating recommendations based on it. 

In [19]:
# Example: Choose a random ISBN from the dataset to use as input
input_isbn = filtered_ratings['isbn'].sample().iloc[0]
input_book_title = get_book_title(input_isbn, books_df_cleaned)

# Generate recommendations
recommended_books = recommend_books(input_isbn, similarity_matrix, books_df_cleaned)

input_book_title, recommended_books

('Rising Tides',
 ['Inner Harbor (Quinn Brothers (Paperback))',
  'Sea Swept (Quinn Brothers (Paperback))',
  'Heart of the Sea (Irish Trilogy)',
  'Tears of the Moon (Irish Trilogy)',
  'Jewels of the Sun (Irish Trilogy)'])

In [18]:
def get_book_info(isbn, books_df):
    """ Get the book title and image URL given an ISBN. """
    book_info = books_df.loc[books_df['ISBN'] == isbn, ['Book-Title', 'Image-URL-L']].iloc[0]
    return book_info

def recommend_books_with_images(isbn, similarity_matrix, books_df, num_recommendations=5):
    """
    Recommend books based on a given ISBN using the item similarity matrix,
    and include their images.
    """
    # Retrieve similar books
    similar_books = similarity_matrix[isbn].sort_values(ascending=False)

    # Remove the book itself from the recommendation
    similar_books = similar_books[similar_books.index != isbn]

    # Get the top N similar books
    top_similar_books = similar_books.head(num_recommendations).index.tolist()

    # Get book titles and image URLs
    recommended_books_info = [get_book_info(book_isbn, books_df) for book_isbn in top_similar_books]

    return recommended_books_info

